<a href="https://colab.research.google.com/github/look4pritam/MachineLearning/blob/master/InsightFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opencv-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!wget https://github.com/leondgarse/Keras_insightface/releases/download/v1.0.0/glint360k_cosface_r100_fp16_0.1.h5 -o model.h5

In [4]:
!mv glint360k_cosface_r100_fp16_0.1.h5 model.h5
!ls -al

total 256160
drwxr-xr-x 1 root root      4096 Apr  2 12:44 .
drwxr-xr-x 1 root root      4096 Apr  2 12:42 ..
drwxr-xr-x 4 root root      4096 Mar 30 13:52 .config
-rw-r--r-- 1 root root 262283552 Dec  7  2021 model.h5
drwxr-xr-x 1 root root      4096 Mar 30 13:53 sample_data


In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
import numpy as np

In [6]:
frozen_out_path = ''
frozen_graph_filename = 'frozen_graph'

In [7]:
model = keras.models.load_model('model.h5')
model.summary()

Model: "ResNet100"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 112, 112, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_49 (ZeroPadding  (None, 114, 114, 3)  0          ['input_2[0][0]']                
 2D)                                                                                              
                                                                                                  
 0_conv (Conv2D)                (None, 112, 112, 64  1728        ['zero_padding2d_49[0][0]']      
                                )                                                         

In [8]:
full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(
    tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))
# Get frozen ConcreteFunction
frozen_func = convert_variables_to_constants_v2(full_model)
frozen_func.graph.as_graph_def()
layers = [op.name for op in frozen_func.graph.get_operations()]
print("-" * 60)
print("Frozen model layers: ")
for layer in layers:
    print(layer)
print("-" * 60)
print("Frozen model inputs: ")
print(frozen_func.inputs)
print("Frozen model outputs: ")
print(frozen_func.outputs)

------------------------------------------------------------
Frozen model layers: 
x
ResNet100/zero_padding2d_49/Pad/paddings
ResNet100/0_conv/Conv2D/ReadVariableOp/resource
ResNet100/0_bn/ReadVariableOp/resource
ResNet100/0_bn/ReadVariableOp_1/resource
ResNet100/0_bn/FusedBatchNormV3/ReadVariableOp/resource
ResNet100/0_bn/FusedBatchNormV3/ReadVariableOp_1/resource
ResNet100/0_PReLU/ReadVariableOp/resource
ResNet100/stack1_block1_shortcut_conv/Conv2D/ReadVariableOp/resource
ResNet100/stack1_block1_shortcut_bn/ReadVariableOp/resource
ResNet100/stack1_block1_shortcut_bn/ReadVariableOp_1/resource
ResNet100/stack1_block1_shortcut_bn/FusedBatchNormV3/ReadVariableOp/resource
ResNet100/stack1_block1_shortcut_bn/FusedBatchNormV3/ReadVariableOp_1/resource
ResNet100/stack1_block1_1_bn/ReadVariableOp/resource
ResNet100/stack1_block1_1_bn/ReadVariableOp_1/resource
ResNet100/stack1_block1_1_bn/FusedBatchNormV3/ReadVariableOp/resource
ResNet100/stack1_block1_1_bn/FusedBatchNormV3/ReadVariableOp_1/re

In [9]:
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir=frozen_out_path,
                  name=f"{frozen_graph_filename}.graphdef",
                  as_text=False)

'frozen_graph.graphdef'

In [10]:
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir=frozen_out_path,
                  name=f"{frozen_graph_filename}.pbtxt",
                  as_text=True)

'frozen_graph.pbtxt'

In [11]:
!ls -al

total 1247092
drwxr-xr-x 1 root root      4096 Apr  2 12:46 .
drwxr-xr-x 1 root root      4096 Apr  2 12:42 ..
drwxr-xr-x 4 root root      4096 Mar 30 13:52 .config
-rw-r--r-- 1 root root 261340612 Apr  2 12:45 frozen_graph.graphdef
-rw-r--r-- 1 root root 753367255 Apr  2 12:46 frozen_graph.pbtxt
-rw-r--r-- 1 root root 262283552 Dec  7  2021 model.h5
drwxr-xr-x 1 root root      4096 Mar 30 13:53 sample_data


In [13]:
import cv2

In [14]:
net = cv2.dnn.readNetFromTensorflow('frozen_graph.graphdef')

In [20]:
outputlayers = net.getUnconnectedOutLayersNames()
print(outputlayers)

('ResNet100/pre_embedding/batchnorm/add_1',)


In [32]:
image = cv2.imread('face.jpg')

In [33]:
net.setInput(cv2.dnn.blobFromImage(image, size=(112, 112), swapRB=True, crop=False))

In [34]:
output = net.forward()

In [38]:
print(output.shape)

(512,)


In [37]:
output = output.reshape(output.shape[1])

In [39]:
output = output / np.linalg.norm(output)

In [40]:
image2 = cv2.imread('face2.jpg')

In [41]:
net.setInput(cv2.dnn.blobFromImage(image2, size=(112, 112), swapRB=True, crop=False))

In [42]:
output2 = net.forward()

In [43]:
output2 = output2.reshape(output2.shape[1])

In [44]:
output2 = output2 / np.linalg.norm(output2)

In [45]:
score = output.dot(output2)

In [46]:
print(score)

0.84101236
